<a href="https://colab.research.google.com/github/gusmaomarcos/portfolio/blob/main/strava_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timezone
from zoneinfo import ZoneInfo

strava = pd.read_csv('/content/drive/MyDrive/Dados/activities.csv')

strava = strava[strava['Activity Type'] == 'Run']

cols = ['Activity Date', 'Activity Name', 'Elapsed Time', 'Distance', 'Max Heart Rate','Average Heart Rate',
        'Max Speed', 'Average Speed', 'Elevation Gain', 'Weather Temperature']

strava = strava[cols]

strava.rename(columns={'Activity Date':'Data', 'Activity Name':'Atividade',
                       'Elapsed Time':'Tempo de Atividade (min)', 'Distance':'Distância (km)',
                       'Max Heart Rate':'BPM Máx', 'Average Heart Rate':'BPM Médio',
                       'Max Speed':'Vel. Máx (km/h)', 'Average Speed':'Vel. Média (km/h)',
                       'Elevation Gain':'Ganho Elev. (m)', 'Weather Temperature':'Temperatura (°C)',
                       }, inplace=True)

strava['Tempo de Atividade (min)'] = strava['Tempo de Atividade (min)'] / 60
strava['Vel. Máx (km/h)'] = strava['Vel. Máx (km/h)'] * 3.672
strava['Vel. Média (km/h)'] = strava['Vel. Média (km/h)'] * 3.672

strava['Duração (min)'] = pd.to_timedelta(strava['Tempo de Atividade (min)'], unit='min')

strava['Pace (min/km)'] = strava['Tempo de Atividade (min)'] / strava['Distância (km)']

strava['Data'] = pd.to_datetime(strava['Data'])

if strava['Data'].dt.tz is None:
    strava['Data'] = strava['Data'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
else:
    strava['Data'] = strava['Data'].dt.tz_convert('America/Sao_Paulo')

average_max_speed_fix = strava[strava['Vel. Máx (km/h)'] < 25]
strava.loc[strava['Vel. Máx (km/h)'] > 25, 'Vel. Máx (km/h)'] = average_max_speed_fix['Vel. Máx (km/h)'].mean()

strava = strava[strava['Pace (min/km)'] <= 9]

strava = strava[strava['Distância (km)'] >= 1.0]

def classify_hr_zone(hr_value):
    if 1 <= hr_value <= 133:
        return 1
    elif 134 <= hr_value <= 147:
        return 2
    elif 148 <= hr_value <= 161:
        return 3
    elif 162 <= hr_value <= 174:
        return 4
    elif 175 <= hr_value <= 205:
        return 5
    else:
        return None

strava['Zona HR'] = strava['BPM Médio'].apply(classify_hr_zone)

corridas = ['Circuito das Estações 2024 - Etapa Verão', 'Corrida da Portela 2024',
         'Circuito das Estações 2024 - Etapa Primavera', 'Bimbo Global Race 2024',
         'Rio-S 21K', 'Circuito das Estações 2024 - Etapa Inverno',
         'Circuito Banco do Brasil 2024', 'Asics Golden Run 2024',
         'Eco Run', 'Maratona do Rio 2024',
         'Night Run 1 - Apollo', 'Circuito das Estações 2024 - Etapa Outono',
         'Circuito do Sol 2024', '1a Corrida de Vera Cruz',
         'Wings for Life World Run - 2019', 'Wings for Life World Run 2024',
         'Circuito das Estações 2019 - Etapa Outono', 'Circuito das Estações 2017 - Etapa Verão',
         'Circuito das Estações 2017 - Etapa Primavera', 'Night Run - Etapa Blue',
         'Circuito das Estações 2017 - Etapa Inverno', 'Night Run Yellow',
         'Circuito das Estações 2017 - Etapa Outono', 'Circuito das Estações 2019 - Etapa Inverno',
         'Circuito Banco do Brasil 2018', '2a Corrida de Vera Cruz',
         'Fla Run 2025'
         ]

strava['Tipo de Atividade'] = 'Treino'

def mark_races(df, races_list):

  df['Tipo de Atividade'] = np.where(df['Atividade'].isin(races_list), 'Corrida', df['Tipo de Atividade'])
  return df

strava = mark_races(strava, corridas)

def mark_long(df,races):

  df['Tipo de Atividade'] = np.where(
      (df['Distância (km)'] > 13) & (~df['Atividade'].isin(races)),
      'Treino Longo',
      df['Tipo de Atividade']
  )
  return df

strava = mark_long(strava, corridas)

strava.to_csv('/content/drive/MyDrive/Dados/strava.csv', index=False)

<ipython-input-5-2d14020716ee>:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  strava['Data'] = pd.to_datetime(strava['Data'])


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer

#data
strava = pd.read_csv('/content/drive/MyDrive/Dados/strava.csv')

#model
strava['Data'] = pd.to_datetime(strava['Data'])
strava['weight'] = np.where(strava['Data'] >= (strava['Data'].max() - pd.DateOffset(months=4)), 2.0, 1.0)

strava['is_race'] = (strava['Atividade'] == 'Corrida').astype(int)
features = ['Distância (km)', 'BPM Médio', 'is_race', 'Temperatura (°C)']
target = 'Pace (min/km)'

scaler = StandardScaler()
strava[features] = scaler.fit_transform(strava[features])

X = strava[features]
y = strava[target]
weights = strava['weight']
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, weights, test_size=0.2)

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

model = LinearRegression()
model.fit(X_train, y_train, sample_weight=w_train)

race_distances = pd.DataFrame({'Distância (km)': [5, 10, 21, 42],
                               'BPM Médio': [190, 185, 180, 170],
                               'is_race': [1, 1, 1, 1],
                               'Temperatura (°C)':[22, 25, 25, 27]})

race_distances_scaled = scaler.transform(race_distances)

predicted_pace = model.predict(race_distances_scaled)

distances = np.array([5, 10, 21.1, 42.195])

finish_times_minutes = predicted_pace * distances

def format_time(minutes):
    hrs = int(minutes // 60)
    mins = int(minutes % 60)
    secs = int((minutes % 1) * 60)
    return f"{hrs:02}:{mins:02}:{secs:02}"

formatted_finish_times = [format_time(time) for time in finish_times_minutes]

results = pd.DataFrame({
    "Distance (km)": distances,
    "Predicted Pace (min/km)": [f"{pace:.2f}" for pace in predicted_pace],
    "Finish Time (HH:MM:SS)": formatted_finish_times
})

results['Predicted Pace (min/km)'] = results['Predicted Pace (min/km)'].astype(float)
results['Finish Time (HH:MM:SS)'] = pd.to_datetime(results['Finish Time (HH:MM:SS)'], format='%H:%M:%S').dt.time

results.to_csv('/content/drive/MyDrive/Dados/results.csv', index=False)

results

<ipython-input-20-42e9c9b9adec>:11: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  strava['Data'] = pd.to_datetime(strava['Data'])


,Distance (km),Predicted Pace (min/km),Finish Time (HH:MM:SS)
0,5.000,4.69,00:23:27
1,10.000,5.10,00:51:01
2,21.100,5.55,01:57:01
3,42.195,6.47,04:33:11
